# 05 - Model Evaluation

This notebook evaluates model performance, generates visualizations, and applies SHAP interpretability analysis.


In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from src.data_processing import DataProcessor
from src.feature_engineering import FeatureEngineer
from src.model_training import ModelTrainer
from src.evaluation import ModelEvaluator

# Prepare data
processor = DataProcessor()
X_train, X_test, y_train, y_test = processor.process(
    '../data/WA_Fn-UseC_-Telco-Customer-Churn.csv',
    target_column='Churn',
    test_size=0.2,
    random_state=42
)

feature_engineer = FeatureEngineer()
X_train_fe, X_test_fe, y_train_balanced, _ = feature_engineer.engineer_features(
    X_train, X_test, y_train, apply_smote=True
)

# Load or train models
trainer = ModelTrainer()
try:
    trainer.load_models('../models')
    print("Models loaded from disk")
except:
    trainer.train_models(X_train_fe, y_train_balanced)
    print("Models trained")

# Evaluate models
evaluator = ModelEvaluator()
results_df = evaluator.evaluate_all_models(trainer.trained_models, X_test_fe, y_test)

# Generate visualizations
evaluator.plot_confusion_matrices(trainer.trained_models, X_test_fe, y_test)
evaluator.plot_roc_curves(trainer.trained_models, X_test_fe, y_test)
evaluator.plot_metrics_comparison(results_df)

# SHAP analysis
best_model_name = results_df.iloc[0]['Model']
best_model = trainer.trained_models[best_model_name]
feature_names = feature_engineer.feature_columns if hasattr(feature_engineer, 'feature_columns') else None
evaluator.apply_shap_analysis(best_model, X_test_fe, best_model_name, feature_names=feature_names, max_samples=100)
